In [11]:

import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import numpy as np
from numpy.linalg import multi_dot, inv
import matplotlib.pyplot as plt
from scipy.sparse import diags


from scipy.io import loadmat
import numpy as np

import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, tril, vstack, diags, hstack
from scipy.sparse.linalg import inv

In [12]:
delta = 3000
T = 0.1
mat = loadmat("dataset1.mat")
num_states = len(mat["r"])
num_samples = num_states//delta
t = np.array(mat["t"][delta::delta])
# H <- [A_inv, C]T
A_inv = tril(np.ones((num_samples, num_samples))).tocsr()
# A_inv = inv(np.tril(np.ones((num_samples, num_samples))))
C = csr_matrix(np.eye(num_samples))
H = vstack([A_inv, C])
H_T = H.T
check_H = H.toarray()
check_H_T = H_T.toarray()
# H = np.concatenate((A_inv,C),axis = 0)
# H_t = csr_matrix(np.transpose(H))
# H_t = np.transpose(H)

# z <- [v, y]T
r = np.array(mat["r"][delta::delta])
l = np.array(mat["l"])
y = l-r
u = mat["v"]
v = np.zeros((num_samples,1))
for i in range(delta, num_states, delta):
    v[int(i/delta-1)] = T*np.sum(u[i-delta:i])
z = np.vstack((v,y))

# W <-[[Q,0],[0, R]]
r_var = mat["r_var"][0,0]
R_inv = diags([1/r_var] * num_samples, format='csr')
# R_inv = inv(np.diag([r_var]*num_samples))
v_var = mat["v_var"][0,0]
Q_inv = diags([1/v_var] * num_samples, format='csr')
# Q_inv = inv(np.diag([v_var]*num_samples))
zero_matrix = csr_matrix((num_samples, num_samples))
W_inv = vstack([hstack([Q_inv, zero_matrix]), hstack([zero_matrix, R_inv])])



# W_inv = csr_matrix(np.block([[Q_inv, np.zeros((num_samples, num_samples))],
#                       [np.zeros((num_samples, num_samples)), R_inv]]))
# W_inv = np.block([[Q_inv, np.zeros((num_samples, num_samples))],
#                     [np.zeros((num_samples, num_samples)), R_inv]])
# sigma
P_hat_inv = H_T.dot(W_inv.dot(H))

P_hat = inv(P_hat_inv)
sigma = np.sqrt(P_hat.diagonal())

In [13]:
A_inv.toarray()

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])